# Rezervacija hotela z vmesnikom za prioritete članov

Ta zvezek prikazuje **vmesnik na osnovi funkcij** z uporabo Microsoft Agent Framework. Nadgrajujemo primer pogojevanega poteka dela z dodajanjem vmesne plasti, ki članom s prioriteto daje posebne privilegije.

## Kaj se boste naučili:
1. **Vmesnik na osnovi funkcij**: Prestrezanje in spreminjanje rezultatov funkcij
2. **Dostop do konteksta**: Branje in spreminjanje `context.result` po izvedbi
3. **Implementacija poslovne logike**: Prednosti članov s prioriteto
4. **Prepis rezultata**: Spreminjanje izidov funkcij glede na status uporabnika
5. **Isti potek dela, različni izidi**: Spremembe vedenja prek vmesne plasti

## Arhitektura poteka dela z vmesnikom:

```
User Input: "I want to book a hotel in Paris"
                    ↓
        [availability_agent]
        - Calls hotel_booking tool
        - 🌟 priority_check middleware intercepts
        - Checks user membership status
        - IF priority + no rooms → Override to available!
        - Returns BookingCheckResult
                    ↓
        Conditional Routing
           /                    \
    [has_availability]    [no_availability]
          ↓                      ↓
    [booking_agent]        [alternative_agent]
    (Priority override!)   (Regular users)
          ↓                      ↓
       [display_result executor]
```

## Ključna razlika od pogojevanega poteka dela:

**Brez vmesnika** (14-conditional-workflow.ipynb):
- Pariz nima sob → Preusmeritev na alternative_agent

**Z vmesnikom** (ta zvezek):
- Navaden uporabnik + Pariz → Ni sob → Preusmeritev na alternative_agent
- Uporabnik s prioriteto + Pariz → 🌟 Vmesnik prepiše! → Na voljo → Preusmeritev na booking_agent

## Predpogoji:
- Nameščen Microsoft Agent Framework
- Razumevanje pogojevanih potekov dela (glej 14-conditional-workflow.ipynb)
- GitHub žeton ali OpenAI API ključ
- Osnovno razumevanje vzorcev vmesnikov


In [1]:
import asyncio
import json
import os
from collections.abc import Awaitable, Callable
from typing import Annotated, Any, Never

from agent_framework import (
    AgentExecutor,
    AgentExecutorRequest,
    AgentExecutorResponse,
    ChatMessage,
    FunctionInvocationContext,
    Role,
    WorkflowBuilder,
    WorkflowContext,
    ai_function,
    executor,
)

# 🤖 GitHub Models or OpenAI client integration
from agent_framework.openai import OpenAIChatClient
from dotenv import load_dotenv
from IPython.display import HTML, display
from pydantic import BaseModel

print("✅ All imports successful!")

✅ All imports successful!


## 1. korak: Določite Pydantic modele za strukturirane izhode

Ti modeli določajo **shemo**, ki jo bodo agenti vrnili. Dodali smo polje `priority_override`, da sledimo, kdaj vmesna programska oprema spremeni rezultat razpoložljivosti.


In [2]:
class BookingCheckResult(BaseModel):
    """Result from checking hotel availability at a destination."""

    destination: str
    has_availability: bool
    message: str
    priority_override: bool = False  # 🆕 NEW! Tracks if middleware overrode the result


class AlternativeResult(BaseModel):
    """Suggested alternative destination when no rooms available."""

    alternative_destination: str
    reason: str


class BookingConfirmation(BaseModel):
    """Booking suggestion when rooms are available."""

    destination: str
    action: str
    message: str


print("✅ Pydantic models defined:")
print("   - BookingCheckResult (availability check with priority_override)")
print("   - AlternativeResult (alternative suggestion)")
print("   - BookingConfirmation (booking confirmation)")

✅ Pydantic models defined:
   - BookingCheckResult (availability check with priority_override)
   - AlternativeResult (alternative suggestion)
   - BookingConfirmation (booking confirmation)


## Korak 2: Določite podatkovno bazo prednostnih članov

Za ta prikaz bomo simulirali podatkovno bazo prednostnih članov. V produkciji bi to poizvedovalo realno podatkovno bazo ali API.

**Prednostni člani:**
- `alice@example.com` - VIP član
- `bob@example.com` - Premium član  
- `priority_user` - Testni račun


In [3]:
# Simulated priority members database
PRIORITY_MEMBERS = {
    "alice@example.com",
    "bob@example.com",
    "priority_user",
}

# Global variable to track current user (in real app, use proper session management)
current_user_id = "regular_user"  # Default: regular user


def set_user(user_id: str):
    """Set the current user for the session."""
    global current_user_id
    current_user_id = user_id
    is_priority = user_id in PRIORITY_MEMBERS
    status = "🌟 PRIORITY MEMBER" if is_priority else "👤 Regular User"

    display(
        HTML(f"""
        <div style='padding: 15px; background: {"linear-gradient(135deg, #FFD700 0%, #FFA500 100%)" if is_priority else "#e3f2fd"}; 
                    border-left: 4px solid {"#FF6B35" if is_priority else "#2196f3"}; border-radius: 4px; margin: 10px 0;'>
            <strong>👤 Current User Set:</strong> {user_id}<br>
            <strong>Status:</strong> {status}
        </div>
    """)
    )


print("✅ Priority members database created")
print(f"   Priority members: {len(PRIORITY_MEMBERS)} users")

✅ Priority members database created
   Priority members: 3 users


## Korak 3: Ustvarite orodje za rezervacijo hotela

Enako kot pogojni potek dela, vendar ga bo zdaj prestregla vmesna programska oprema!


In [4]:
@ai_function(description="Check hotel room availability for a destination city")
def hotel_booking(destination: Annotated[str, "The destination city to check for hotel rooms"]) -> str:
    """
    Simulates checking hotel room availability.
    
    Returns JSON string with availability status.
    """
    display(
        HTML(f"""
        <div style='padding: 15px; background: #e3f2fd; border-left: 4px solid #2196f3; border-radius: 4px; margin: 10px 0;'>
            <strong>🔍 Tool Invoked:</strong> hotel_booking("{destination}")
        </div>
    """)
    )

    # Simulate availability check
    cities_with_rooms = ["stockholm", "seattle", "tokyo", "london", "amsterdam"]
    has_rooms = destination.lower() in cities_with_rooms

    result = {"has_availability": has_rooms, "destination": destination}

    return json.dumps(result)


print("✅ hotel_booking tool created with @ai_function decorator")

✅ hotel_booking tool created with @ai_function decorator


## Korak 4: 🌟 Ustvarite vmesnik za preverjanje prioritete (KLJUČNA FUNKCIJA!)

To je **osnovna funkcionalnost** tega zapiska. Vmesnik:

1. **Ujeta** klic funkcije hotel_booking
2. **Izvede** funkcijo običajno s klicem `next(context)`
3. **Preveri** rezultat v `context.result`
4. **Prepiše** rezultat, če je uporabnik prioriteten in ni na voljo sob
5. **Vrne** spremenjeni rezultat nazaj agentu

**Ključni vzorec:**
```python
async def my_middleware(context, next):
    await next(context)  # Izvedi funkcijo
    # Zdaj context.result vsebuje izhod funkcije
    if some_condition:
        context.result = new_value  # Prepiši!
```


In [5]:
async def priority_check_middleware(
    context: FunctionInvocationContext,
    next: Callable[[FunctionInvocationContext], Awaitable[None]],
) -> None:
    """
    Function middleware that overrides hotel_booking results for priority members.
    
    Workflow:
    1. Let the function execute normally
    2. Check if user is a priority member
    3. If priority + no availability → Override to make rooms available!
    4. Agent will then route to booking path instead of alternative path
    """
    function_name = context.function.name

    display(
        HTML(f"""
        <div style='padding: 12px; background: #fff3e0; border-left: 4px solid #ff9800; border-radius: 4px; margin: 10px 0;'>
            <strong>🔄 Middleware:</strong> Intercepting {function_name}...
        </div>
    """)
    )

    # Execute the original function
    await next(context)

    # Now inspect and potentially modify the result
    if context.result and function_name == "hotel_booking":
        result_data = json.loads(context.result)
        destination = result_data.get("destination", "")
        has_availability = result_data.get("has_availability", False)

        # Check if user is priority member
        is_priority = current_user_id in PRIORITY_MEMBERS

        # Override logic: Priority member + no availability → Make available!
        if is_priority and not has_availability:
            display(
                HTML(f"""
                <div style='padding: 20px; background: linear-gradient(135deg, #FFD700 0%, #FFA500 100%); 
                            border-radius: 8px; margin: 10px 0; box-shadow: 0 4px 12px rgba(255,165,0,0.4);'>
                    <h3 style='margin: 0 0 10px 0; color: #333;'>🌟 PRIORITY OVERRIDE ACTIVATED! 🌟</h3>
                    <p style='margin: 0; color: #555; line-height: 1.6;'>
                        <strong>User:</strong> {current_user_id}<br>
                        <strong>Status:</strong> VIP Priority Member<br>
                        <strong>Action:</strong> Overriding "No Availability" for {destination}<br>
                        <strong>Result:</strong> ✅ Rooms now available for priority booking!
                    </p>
                </div>
            """)
            )

            # Override the result!
            result_data["has_availability"] = True
            result_data["priority_override"] = True
            context.result = json.dumps(result_data)

        elif not has_availability:
            display(
                HTML(f"""
                <div style='padding: 12px; background: #ffebee; border-left: 4px solid #f44336; border-radius: 4px; margin: 10px 0;'>
                    <strong>ℹ️ Middleware:</strong> No priority override (user: {current_user_id})
                </div>
            """)
            )


print("✅ priority_check_middleware created")
print("   - Intercepts hotel_booking function")
print("   - Overrides availability for priority members")

✅ priority_check_middleware created
   - Intercepts hotel_booking function
   - Overrides availability for priority members


## Korak 5: Določite funkcije pogojev za usmerjanje

Enake funkcije pogojev kot v pogojnem delovnem toku - pregledajo strukturiran izhod za določitev usmerjanja.


In [6]:
def has_availability_condition(message: Any) -> bool:
    """Condition for routing when hotels ARE available (including priority overrides!)."""
    if not isinstance(message, AgentExecutorResponse):
        return True

    try:
        result = BookingCheckResult.model_validate_json(message.agent_run_response.text)

        # Check if this was a priority override
        override_indicator = " 🌟" if result.priority_override else ""

        display(
            HTML(f"""
            <div style='padding: 12px; background: #c8e6c9; border-left: 4px solid #4caf50; border-radius: 4px; margin: 10px 0;'>
                <strong>✅ Condition Check:</strong> has_availability = <strong>{result.has_availability}</strong> for {result.destination}{override_indicator}
            </div>
        """)
        )

        return result.has_availability
    except Exception as e:
        display(
            HTML(f"""
            <div style='padding: 12px; background: #ffcdd2; border-left: 4px solid #f44336; border-radius: 4px; margin: 10px 0;'>
                <strong>⚠️  Error:</strong> {str(e)}
            </div>
        """)
        )
        return False


def no_availability_condition(message: Any) -> bool:
    """Condition for routing when hotels are NOT available."""
    if not isinstance(message, AgentExecutorResponse):
        return False

    try:
        result = BookingCheckResult.model_validate_json(message.agent_run_response.text)

        display(
            HTML(f"""
            <div style='padding: 12px; background: #ffecb3; border-left: 4px solid #ff9800; border-radius: 4px; margin: 10px 0;'>
                <strong>❌ Condition Check:</strong> no_availability for {result.destination}
            </div>
        """)
        )

        return not result.has_availability
    except Exception:
        return False


print("✅ Condition functions defined")

✅ Condition functions defined


## Korak 6: Ustvari izvrševalec po meri za prikaz

Enak izvrševalec kot prej - prikazuje končni izhod delovnega toka.


In [7]:
@executor(id="display_result")
async def display_result(response: AgentExecutorResponse, ctx: WorkflowContext[Never, str]) -> None:
    """Display the final result as workflow output."""
    display(
        HTML("""
        <div style='padding: 15px; background: #f3e5f5; border-left: 4px solid #9c27b0; border-radius: 4px; margin: 10px 0;'>
            <strong>📤 Display Executor:</strong> Yielding workflow output
        </div>
    """)
    )

    await ctx.yield_output(response.agent_run_response.text)


print("✅ display_result executor created")

✅ display_result executor created


## Korak 7: Naložite spremenljivke okolja

Konfigurirajte odjemalca LLM (GitHub Models ali OpenAI).


In [8]:
# Load environment variables
load_dotenv()

# Check for GitHub Models or OpenAI
chat_client = OpenAIChatClient(base_url=os.environ.get(
    "GITHUB_ENDPOINT"), api_key=os.environ.get("GITHUB_TOKEN"), model_id="gpt-4o")


## Korak 8: Ustvarjanje AI agentov z vmesno programsko opremo

**KLJUČNA RAZLIKA:** Ko ustvarjamo availability_agent, posredujemo parameter `middleware`!

Tako vbrizgamo priority_check_middleware v cevovod klica funkcij agenta.


In [9]:
# Agent 1: Check availability with tool + middleware
availability_agent = AgentExecutor(
    chat_client.create_agent(
        instructions=(
            "You are a hotel booking assistant that checks room availability. "
            "Use the hotel_booking tool to check if rooms are available at the destination. "
            "Return JSON with fields: destination (string), has_availability (bool), message (string), "
            "and priority_override (bool, true if priority member got special access). "
            "The message should summarize the availability status and mention if priority override occurred."
        ),
        tools=[hotel_booking],
        response_format=BookingCheckResult,
        middleware=[priority_check_middleware],  # 🌟 MIDDLEWARE INJECTION!
    ),
    id="availability_agent",
)

# Agent 2: Suggest alternative (when no rooms)
alternative_agent = AgentExecutor(
    chat_client.create_agent(
        instructions=(
            "You are a helpful travel assistant. When a user cannot find hotels in their requested city, "
            "suggest an alternative nearby city that has availability. "
            "Return JSON with fields: alternative_destination (string) and reason (string). "
            "Make your suggestion sound appealing and helpful."
        ),
        response_format=AlternativeResult,
    ),
    id="alternative_agent",
)

# Agent 3: Suggest booking (when rooms available)
booking_agent = AgentExecutor(
    chat_client.create_agent(
        instructions=(
            "You are a booking assistant. The user has found available hotel rooms. "
            "Encourage them to book by highlighting the destination's appeal. "
            "If priority_override is true in the input, mention that they received priority member access. "
            "Return JSON with fields: destination (string), action (string), and message (string). "
            "The action should be 'book_now' and message should be encouraging."
        ),
        response_format=BookingConfirmation,
    ),
    id="booking_agent",
)

display(
    HTML("""
    <div style='padding: 15px; background: #e3f2fd; border-left: 4px solid #2196f3; border-radius: 4px; margin: 10px 0;'>
        <strong>✅ Created 3 Agents:</strong>
        <ul style='margin: 10px 0 0 0;'>
            <li><strong>availability_agent</strong> - WITH priority_check_middleware 🌟</li>
            <li><strong>alternative_agent</strong> - Suggests alternative cities</li>
            <li><strong>booking_agent</strong> - Encourages booking</li>
        </ul>
    </div>
""")
)

## Korak 9: Zgradi delovni potek

Ista struktura delovnega toka kot prej - pogojno usmerjanje glede na razpoložljivost.


In [10]:
# Build the workflow with conditional routing
workflow = (
    WorkflowBuilder()
    .set_start_executor(availability_agent)
    # NO AVAILABILITY PATH
    .add_edge(availability_agent, alternative_agent, condition=no_availability_condition)
    .add_edge(alternative_agent, display_result)
    # HAS AVAILABILITY PATH (can be triggered by middleware override!)
    .add_edge(availability_agent, booking_agent, condition=has_availability_condition)
    .add_edge(booking_agent, display_result)
    .build()
)

display(
    HTML("""
    <div style='padding: 20px; background: linear-gradient(135deg, #667eea 0%, #764ba2 100%); color: white; border-radius: 8px; margin: 10px 0;'>
        <h3 style='margin: 0 0 15px 0;'>✅ Workflow Built Successfully!</h3>
        <p style='margin: 0; line-height: 1.6;'>
            <strong>Conditional Routing (Middleware-Aware):</strong><br>
            • If <strong>NO availability</strong> → alternative_agent → display_result<br>
            • If <strong>availability</strong> (or 🌟 <strong>priority override</strong>) → booking_agent → display_result
        </p>
    </div>
""")
)

## Korak 10: Testni primer 1 - Navaden uporabnik v Parizu (Brez preglasitve)

Navaden uporabnik poskuša rezervirati Pariz → Ni sob → Preusmeritev na alternative_agent


In [11]:
# Set as regular user
set_user("regular_user")

display(
    HTML("""
    <div style='padding: 20px; background: #fff3e0; border-left: 4px solid #ff9800; border-radius: 8px; margin: 20px 0;'>
        <h3 style='margin: 0 0 10px 0; color: #e65100;'>🧪 TEST CASE 1: Regular User + Paris</h3>
        <p style='margin: 0;'><strong>Expected:</strong> No rooms → No middleware override → Alternative suggestion</p>
    </div>
""")
)

# Create request
request_regular = AgentExecutorRequest(
    messages=[ChatMessage(Role.USER, text="I want to book a hotel in Paris")], should_respond=True
)

# Run workflow
events_regular = await workflow.run(request_regular)
outputs_regular = events_regular.get_outputs()

# Display results
if outputs_regular:
    result_regular = AlternativeResult.model_validate_json(outputs_regular[0])

    display(
        HTML(f"""
        <div style='padding: 25px; background: #fff; border: 2px solid #ff9800; border-radius: 12px; margin: 20px 0;'>
            <h3 style='margin: 0 0 15px 0; color: #e65100;'>📊 RESULT (Regular User)</h3>
            <div style='background: #fff3e0; padding: 20px; border-radius: 8px;'>
                <p style='margin: 0 0 10px 0;'><strong>Status:</strong> ❌ No rooms in Paris</p>
                <p style='margin: 0 0 10px 0;'><strong>Middleware:</strong> No priority override (regular user)</p>
                <p style='margin: 0 0 10px 0;'><strong>Alternative:</strong> 🏨 {result_regular.alternative_destination}</p>
                <p style='margin: 0;'><strong>Reason:</strong> {result_regular.reason}</p>
            </div>
        </div>
    """)
    )

## Korak 11: Testni primer 2 - 🌟 Prioritetni uporabnik v Parizu (Z OVERRIDE!)

Član s prednostjo poskuša rezervirati Pariz → Sprva ni sob → 🌟 Middleware prevlada! → Usmeri na booking_agent

**To je ključna demonstracija moči middleware!**


In [12]:
# Set as priority user
set_user("priority_user")

display(
    HTML("""
    <div style='padding: 20px; background: linear-gradient(135deg, #FFD700 0%, #FFA500 100%); border-radius: 8px; margin: 20px 0;'>
        <h3 style='margin: 0 0 10px 0; color: #333;'>🧪 TEST CASE 2: 🌟 Priority User + Paris</h3>
        <p style='margin: 0; color: #555;'><strong>Expected:</strong> No rooms → 🌟 MIDDLEWARE OVERRIDE → Rooms available → Booking suggestion!</p>
    </div>
""")
)

# Create request
request_priority = AgentExecutorRequest(
    messages=[ChatMessage(Role.USER, text="I want to book a hotel in Paris")], should_respond=True
)

# Run workflow
events_priority = await workflow.run(request_priority)
outputs_priority = events_priority.get_outputs()

# Display results
if outputs_priority:
    result_priority = BookingConfirmation.model_validate_json(outputs_priority[0])

    display(
        HTML(f"""
        <div style='padding: 25px; background: linear-gradient(135deg, #FFD700 0%, #FFA500 100%); border-radius: 12px; 
                    box-shadow: 0 8px 16px rgba(255,165,0,0.4); margin: 20px 0;'>
            <h3 style='margin: 0 0 15px 0; color: #333;'>🏆 RESULT (Priority Member) 🌟</h3>
            <div style='background: white; padding: 20px; border-radius: 8px;'>
                <p style='margin: 0 0 10px 0; font-size: 16px;'><strong>Status:</strong> ✅ Rooms Available (Priority Override!)</p>
                <p style='margin: 0 0 10px 0; font-size: 16px;'><strong>Middleware:</strong> 🌟 OVERRIDE ACTIVATED!</p>
                <p style='margin: 0 0 10px 0; font-size: 16px;'><strong>Destination:</strong> 🏨 {result_priority.destination}</p>
                <p style='margin: 0 0 10px 0; font-size: 16px;'><strong>Action:</strong> {result_priority.action}</p>
                <p style='margin: 0; font-size: 14px; color: #666;'><strong>Message:</strong> {result_priority.message}</p>
                <div style='margin-top: 15px; padding: 15px; background: #fff3cd; border-radius: 6px; border-left: 4px solid #FF6B35;'>
                    <strong>💡 What Just Happened:</strong><br>
                    1. hotel_booking tool returned "no availability"<br>
                    2. priority_check_middleware intercepted the result<br>
                    3. Middleware checked user status: priority_user ✅<br>
                    4. Middleware OVERRODE the result to "available"<br>
                    5. Workflow routed to booking_agent instead of alternative_agent!
                </div>
            </div>
        </div>
    """)
    )

## Korak 12: Testni primer 3 - Prednostni uporabnik v Stockholmu (že na voljo)

Prednostni uporabnik poskusi Stockholm → Prosti prostori → Ni potrebe po preglasitvi → Usmerjanje na booking_agent

To kaže, da vmesnik deluje le, ko je to potrebno!


In [13]:
# Priority user is still set from previous test

display(
    HTML("""
    <div style='padding: 20px; background: #e8f5e9; border-left: 4px solid #4caf50; border-radius: 8px; margin: 20px 0;'>
        <h3 style='margin: 0 0 10px 0; color: #1b5e20;'>🧪 TEST CASE 3: Priority User + Stockholm</h3>
        <p style='margin: 0;'><strong>Expected:</strong> Rooms available → No override needed → Booking suggestion</p>
    </div>
""")
)

# Create request
request_stockholm = AgentExecutorRequest(
    messages=[ChatMessage(Role.USER, text="I want to book a hotel in Stockholm")], should_respond=True
)

# Run workflow
events_stockholm = await workflow.run(request_stockholm)
outputs_stockholm = events_stockholm.get_outputs()

# Display results
if outputs_stockholm:
    result_stockholm = BookingConfirmation.model_validate_json(outputs_stockholm[0])

    display(
        HTML(f"""
        <div style='padding: 25px; background: linear-gradient(135deg, #4caf50 0%, #8bc34a 100%); color: white; border-radius: 12px; 
                    box-shadow: 0 4px 12px rgba(76,175,80,0.3); margin: 20px 0;'>
            <h3 style='margin: 0 0 15px 0;'>🏆 RESULT (Priority User - No Override Needed)</h3>
            <div style='background: white; color: #333; padding: 20px; border-radius: 8px;'>
                <p style='margin: 0 0 10px 0; font-size: 16px;'><strong>Status:</strong> ✅ Rooms Available (Natural)</p>
                <p style='margin: 0 0 10px 0; font-size: 16px;'><strong>Middleware:</strong> No override needed</p>
                <p style='margin: 0 0 10px 0; font-size: 16px;'><strong>Destination:</strong> 🏨 {result_stockholm.destination}</p>
                <p style='margin: 0 0 10px 0; font-size: 16px;'><strong>Action:</strong> {result_stockholm.action}</p>
                <p style='margin: 0; font-size: 14px; color: #666;'><strong>Message:</strong> {result_stockholm.message}</p>
                <div style='margin-top: 15px; padding: 15px; background: #e8f5e9; border-radius: 6px; border-left: 4px solid #4caf50;'>
                    <strong>💡 Middleware Behavior:</strong><br>
                    • hotel_booking returned "available" naturally<br>
                    • Middleware saw available = true → No override needed<br>
                    • Workflow proceeded normally to booking_agent
                </div>
            </div>
        </div>
    """)
    )

## Ključne ugotovitve in koncepti middleware-a

### ✅ Kaj ste se naučili:

#### **1. Vzorec middleware na osnovi funkcij**

Middleware prestreže klice funkcij z uporabo preproste async funkcije:

```python
async def my_middleware(
    context: FunctionInvocationContext,
    next: Callable,
) -> None:
    # Preden se funkcija izvede
    print("Intercepting...")
    
    # Izvedi funkcijo
    await next(context)
    
    # Po izvedbi funkcije - preglej rezultat
    if context.result:
        # Po potrebi spremeni rezultat
        context.result = modified_value
```

#### **2. Dostop do konteksta in prevlada rezultata**

- `context.function` - Dostop do klicane funkcije
- `context.arguments` - Branje argumentov funkcije
- `context.kwargs` - Dostop do dodatnih parametrov
- `await next(context)` - Izvedba funkcije
- `context.result` - Branje/sprememba izhoda funkcije

#### **3. Implementacija poslovne logike**

Naš middleware implementira prednostne koristi članov:
- **Običajni uporabniki**: Brez sprememb, standardni potek dela
- **Prioritetni uporabniki**: Prevlada "brez razpoložljivosti" → "na voljo"
- **Pogojna logika**: Prevladovanje samo, ko je potrebno

#### **4. Enak potek dela, različni izidi**

Moč middleware-a:
- ✅ Brez sprememb v strukturi poteka dela
- ✅ Brez sprememb v funkciji orodja
- ✅ Brez sprememb v pogojni usmeritvi
- ✅ Samo middleware → Različno vedenje!

### 🚀 Praktične uporabe:

1. **VIP/Premijske funkcije**
   - Prevladovanje omejitev hitrosti za premijske uporabnike
   - Zagotavljanje prednostnega dostopa do virov
   - Dinamična odklepanja premijskih funkcij

2. **A/B testiranje**
   - Usmerjanje uporabnikov na različne implementacije
   - Preizkušanje novih funkcij z določenimi uporabniki
   - Postopni lansiranja funkcij

3. **Varnost in skladnost**
   - Revizija klicev funkcij
   - Blokiranje občutljivih operacij
   - Izvajanje poslovnih pravil

4. **Optimizacija zmogljivosti**
   - Predpomnjenje rezultatov za določene uporabnike
   - Preskakovanje dragih operacij, kadar je mogoče
   - Dinamična dodelitev virov

5. **Obravnava napak in ponovni poskusi**
   - Lov in elegantno ravnanje z napakami
   - Implementacija logike ponovnih poskusov
   - Prehod na alternativne implementacije

6. **Dnevniški zapisi in nadzor**
   - Spremljanje časa izvajanja funkcij
   - Beleženje parametrov in rezultatov
   - Nadzor vzorcev uporabe

### 🔑 Ključne razlike glede na dekoratorje:

| Funkcija | Dekorator | Middleware |
|---------|-----------|------------|
| **Obseg** | Ena funkcija | Vse funkcije v agentu |
| **Prilagodljivost** | Fiksna pri definiciji | Dinamična med izvajanjem |
| **Kontekst** | Omejen | Celoten kontekst agenta |
| **Sestava** | Več dekoratorjev | Pipeline middleware-a |
| **Agent-zaveden** | Ne | Da (dostop do stanja agenta) |

### 📚 Kdaj uporabljati middleware:

✅ **Middleware uporabite, kadar:**
- Potrebujete spremeniti vedenje glede na stanje uporabnika/seje
- Želite uporabiti logiko na več funkcijah
- Potrebujete dostop do konteksta na ravni agenta
- Implementirate prečne skrbi (dnevniški zapisi, avtentikacija, ipd.)

❌ **Middleware ne uporabljajte, kadar:**
- Preprosta validacija vhodnih podatkov (uporabite Pydantic)
- Logika specifična za funkcijo (naj ostane v funkciji)
- Enkratne spremembe (spreminjajte le funkcijo)

### 🎓 Napredni vzorci:

```python
# Več vmesnih programov (pomemben je vrstni red izvajanja!)
middleware=[
    logging_middleware,      # Najprej dnevniki
    auth_middleware,         # Nato preveri avtentikacijo
    cache_middleware,        # Nato preveri predpomnilnik
    rate_limit_middleware,   # Nato omejitve hitrosti
    priority_check_middleware  # Nazadnje preverjanje prednosti
]

# Pogojno izvajanje vmesnega programa
async def conditional_middleware(context, next):
    if should_execute(context):
        await next(context)
        # Spremeni rezultat
    else:
        # Popolnoma preskoči izvajanje
        context.result = cached_value
```

### 🔗 Povezani koncepti:

- **Agent Middleware**: Prestreza klice agent.run()
- **Middleware funkcij**: Prestreza klice funkcij orodij (to smo uporabili!)
- **Pipeline middleware-a**: Veriga middleware-ev, ki se izvajajo po vrsti
- **Propagacija konteksta**: Prenos stanja skozi verigo middleware-ev


---

<!-- CO-OP TRANSLATOR DISCLAIMER START -->
**Omejitev odgovornosti**:
Ta dokument je bil preveden z uporabo storitve za avtomatski prevod AI [Co-op Translator](https://github.com/Azure/co-op-translator). Čeprav si prizadevamo za natančnost, vas opozarjamo, da avtomatizirani prevodi lahko vsebujejo napake ali netočnosti. Izvirni dokument v izvorni jezik velja za avtoritativni vir. Za kritične informacije priporočamo strokovni prevod, opravljen s strani ljudi. Ne odgovarjamo za morebitna nesporazume ali napačne interpretacije, ki izhajajo iz uporabe tega prevoda.
<!-- CO-OP TRANSLATOR DISCLAIMER END -->
